https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md


In [4]:
import openai
import os

In [2]:
from openai import OpenAI

In [7]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [14]:
response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [17]:
response.choices[0].message.content

NameError: name 'response' is not defined

In [1]:
!pip install requests

In [69]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']


    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

ok


## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

In [3]:
!pip install elasticsearch

  Using cached elasticsearch-8.14.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached elastic_transport-8.13.1-py3-none-any.whl.metadata (3.7 kB)
Using cached elasticsearch-8.14.0-py3-none-any.whl (480 kB)
Using cached elastic_transport-8.13.1-py3-none-any.whl (64 kB)


## Q3. Searching

In [6]:
from elasticsearch import Elasticsearch

In [7]:
es_client = Elasticsearch('http://localhost:9200') 

In [10]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
        }
    }
}

In [11]:
index_name = "course-questions"


In [12]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.93it/s]


In [17]:
query = "How do I execute a command in a running docker container?"

In [29]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

In [30]:
response = es_client.search(index=index_name, body=search_query)


In [31]:
response

ObjectApiResponse({'took': 841, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 865, 'relation': 'eq'}, 'max_score': 84.050095, 'hits': [{'_index': 'course-questions', '_id': 'dPeUUpABHqifPMsd_J0-', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'gveUUpABHqifPMsdz5ty', '_score': 75.54128, '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker

In [32]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'dPeUUpABHqifPMsd_J0-',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'gveUUpABHqifPMsdz5ty',
  '_score': 75.54128,
  '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\n

## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

- How do I debug a docker container?
- How do I copy files from a different folder into docker container’s working directory?
- How do Lambda container images work?
- How can I annotate a graph?


In [46]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


In [47]:
es_client.indices.create(index="ml_questions", body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [ml_questions/CvVnKvOtTTmRPQIX6Luohw] already exists')

In [70]:
for doc in tqdm(documents):
    es_client.index(index="ml_questions", document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:22<00:00, 42.84it/s]


In [71]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [72]:
response = es_client.search(index="ml_questions", body=search_query)


In [74]:
response['hits']['hits'][2]

{'_index': 'ml_questions',
 '_id': '_Pe1UpABHqifPMsdcaST',
 '_score': 49.938507,
 '_source': {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from a different folder into docker container’s working directory?',
  'course': 'machine-learning-zoomcamp'}}

In [65]:
[u['course'] for u in documents_raw]

['data-engineering-zoomcamp', 'machine-learning-zoomcamp', 'mlops-zoomcamp']

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

In [75]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [85]:
context = ""

In [86]:
for u in response['hits']['hits']:
    doc = u['_source']
    # print(doc)
    s = f"Q: {doc['question']}\nA: {doc['text']}"
    s.strip()
    s += '\n\n'
    context = context + s
context.strip()

'Q: How do I debug a docker container?\nA: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\nA: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\nA: You can copy files from your local machine into a Docker container using the docker cp comman

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

In [87]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [88]:
prompt = prompt_template.format(question=query, context=context).strip()

In [89]:
len(prompt)

2714

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

In [94]:
!pip install tiktoken

Let's calculate the number of tokens in our query:

In [96]:
import tiktoken

ModuleNotFoundError: No module named 'tiktoken'

In [97]:
!python --version

Python 3.12.3


encoding = tiktoken.encoding_for_model("gpt-4o")


Note: to decode back a token into a word, you can use the decode_single_token_bytes function:

encoding.decode_single_token_bytes(63842)